# Mesh generation for tutorial 06, case 7a

This file generates the mesh which is used in the following examples:
* 7a_stokes_dirichlet_control

The test case is from section 5 of
```
F. Negri, A. Manzoni and G. Rozza. Reduced basis approximation of parametrized optimal flow control problems for the Stokes equations. Computer and Mathematics with Applications, 69(4):319-336, 2015.
```

In [ ]:
import pygmsh
from dolfinx import Mesh, MeshFunction, MeshValueCollection, MPI
from dolfinx.cpp.mesh import CellType, GhostMode
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot

### Geometrical parameters and related quantities

In [ ]:
L1 = 0.9
L2 = 0.35
L3 = 0.55
L4 = 0.2
H = 1.0
r = 0.1
lcar = 0.025

### Create mesh

In [ ]:
geom = pygmsh.built_in.Geometry()
p0 = geom.add_point([0.0, 0.0, 0.0], lcar)
p1 = geom.add_point([L1, 0.0, 0.0], lcar)
p2 = geom.add_point([L1 + L2, 0.0, 0.0], lcar)
p3 = geom.add_point([L1 + L2 + L3, 0.0, 0.0], lcar)
p4 = geom.add_point([L1 + L2 + L3 + L4, 0.0, 0.0], lcar)
p5 = geom.add_point([L1 + L2 + L3 + L4, H, 0.0], lcar)
p6 = geom.add_point([L1 + L2 + L3, H, 0.0], lcar)
p7 = geom.add_point([L1 + L2, H, 0.0], lcar)
p8 = geom.add_point([L1, H, 0.0], lcar)
p9 = geom.add_point([0.0, H, 0.0], lcar)
p10 = geom.add_point([L1, H / 2, 0.0], lcar)
p11 = geom.add_point([L1, H / 2 + r, 0.0], lcar)
p12 = geom.add_point([L1, H / 2 - r, 0.0], lcar)
p13 = geom.add_point([L1 + L2, H / 2 - r, 0.0], lcar)
p14 = geom.add_point([L1 + L2 + L3, H / 2 - 3 * r, 0.0], lcar)
p15 = geom.add_point([L1 + L2 + L3, H / 2 + 3 * r, 0.0], lcar)
p16 = geom.add_point([L1 + L2, H / 2 + r, 0.0], lcar)
l0 = geom.add_line(p0, p1)
l1 = geom.add_line(p1, p2)
l2 = geom.add_line(p2, p3)
l3 = geom.add_line(p3, p4)
l4 = geom.add_line(p4, p5)
l5 = geom.add_line(p5, p6)
l6 = geom.add_line(p6, p7)
l7 = geom.add_line(p7, p8)
l8 = geom.add_line(p8, p9)
l9 = geom.add_line(p9, p0)
l10 = geom.add_line(p12, p13)
l11 = geom.add_line(p13, p14)
l12 = geom.add_line(p14, p15)
l13 = geom.add_line(p15, p16)
l14 = geom.add_line(p16, p11)
l15 = geom.add_line(p13, p16)
l16 = geom.add_line(p1, p12)
l17 = geom.add_line(p11, p8)
l18 = geom.add_line(p2, p13)
l19 = geom.add_line(p16, p7)
l20 = geom.add_line(p3, p14)
l21 = geom.add_line(p15, p6)
c0 = geom.add_circle_arc(p11, p10, p12)
geom.add_physical([l9], label=1)
geom.add_physical([l0, l1, l2, l3, l5, l6, l7, l8], label=2)
geom.add_physical([l4], label=3)
geom.add_physical([l10, l14], label=4)
geom.add_physical([c0, l15], label=5)
line_loop_subdomain1 = geom.add_line_loop([l0, l16, -c0, l17, l8, l9])
line_loop_subdomain2a = geom.add_line_loop([l1, l18, -l10, -l16])
line_loop_subdomain2b = geom.add_line_loop([l7, -l17, -l14, l19])
line_loop_subdomain3a = geom.add_line_loop([l2, l20, -l11, -l18])
line_loop_subdomain3b = geom.add_line_loop([l6, -l19, -l13, l21])
line_loop_subdomain3c = geom.add_line_loop([l3, l4, l5, -l21, -l12, -l20])
line_loop_subdomain4 = geom.add_line_loop([l11, l12, l13, -l15])
subdomain1 = geom.add_plane_surface(line_loop_subdomain1)
subdomain2a = geom.add_plane_surface(line_loop_subdomain2a)
subdomain2b = geom.add_plane_surface(line_loop_subdomain2b)
subdomain3a = geom.add_plane_surface(line_loop_subdomain3a)
subdomain3b = geom.add_plane_surface(line_loop_subdomain3b)
subdomain3c = geom.add_plane_surface(line_loop_subdomain3c)
subdomain4 = geom.add_plane_surface(line_loop_subdomain4)
geom.add_physical(subdomain1, label=11)
geom.add_physical([subdomain2a, subdomain2b], label=12)
geom.add_physical([subdomain3a, subdomain3b, subdomain3c], label=13)
geom.add_physical(subdomain4, label=14)
pygmsh_mesh = pygmsh.generate_mesh(geom)
mesh = Mesh(MPI.comm_world, CellType.triangle, pygmsh_mesh.points[:, :2],
            pygmsh_mesh.cells_dict["triangle"], [], GhostMode.none)

In [ ]:
plot(mesh)

### Create subdomains

In [ ]:
subdomains_mvc = MeshValueCollection("int", mesh, mesh.topology.dim, pygmsh_mesh.cells_dict["triangle"],
                                     pygmsh_mesh.cell_data_dict["gmsh:physical"]["triangle"] - 10)
subdomains = MeshFunction("int", mesh, subdomains_mvc, 0)

### Create boundaries

In [ ]:
boundaries_mvc = MeshValueCollection("int", mesh, mesh.topology.dim - 1, pygmsh_mesh.cells_dict["line"],
                                     pygmsh_mesh.cell_data_dict["gmsh:physical"]["line"])
boundaries = MeshFunction("int", mesh, boundaries_mvc, 0)

### Save

In [ ]:
with XDMFFile(MPI.comm_world, "vorticity_reduction.xdmf") as output:
    output.write(mesh)
with XDMFFile(MPI.comm_world, "vorticity_reduction_subdomains.xdmf") as output:
    output.write(subdomains)
with XDMFFile(MPI.comm_world, "vorticity_reduction_boundaries.xdmf") as output:
    output.write(boundaries)